# Return behaviour in geneartor 

In [1]:
def f():
    exception_value = "hoge"
    while True:
        yield 1
        return exception_value  # Same with raising StopIteration(exception_value)

_f = f()
print(next(_f))
print(next(_f))

1


StopIteration: hoge

# Generator instantiation executes nothing
Instantiate a generator only creates an object and stops there.

In [10]:
def f():
    print("generator started")
    event = 99999
    while True:
        event = (yield event) # yield statement is evaluated to None with next()
        print("event is {}".format(event))
        
g = f()  # Does not print "generator started"

## send(value) behavior after instantiation
[PEP 342 Specification: Sending Values into Generators](https://www.python.org/dev/peps/pep-0342/#specification-sending-values-into-generators)

> Because generator-iterators begin execution at the top of the generator's function body, there is no yield expression to receive a value when the generator has just been created. Therefore, **calling send() with a non-None argument is prohibited when the generator iterator has just started**, and a TypeError is raised if this occurs (presumably due to a logic error of some kind). Thus, before you can communicate with a coroutine you must **first call next() or send(None)** to advance its execution to the first yield expression.

In [11]:
g.send(5)

TypeError: can't send non-None value to a just-started generator

In [12]:
g.send(None)
g.send(5)

generator started
event is 5


5

## next() is same with send(None) 
[generator.__next__()](https://docs.python.org/3/reference/expressions.html#generator.__next__)

> When a generator function is resumed with a __next__() method, the **current yield expression always evaluates to None**. 

In [14]:
# Will not return the value 5 sent and set in the previous send(5)
next(g)

event is None


## close() behaviour

[generator.close()](https://docs.python.org/3/reference/expressions.html#generator.close)

> Raises a GeneratorExit at the point where the generator function was paused. If the generator function then exits gracefully, is already closed, or raises GeneratorExit (by not catching the exception), close returns to its caller. 

In [16]:
g.close()
print(next(g))  # Raises StopIteration as it has been already terminated.

StopIteration: 

# Demestify yield statement

In [11]:
def f():
    event_expression = 99999
    while True:
        # In the execution cycle c(n) upon the caller invocation send(value(n)) at t=n.
        # "yield event_expression" is evaluated and returned to the caller, but not yet with "value=()"
        # c(n) terminates and the generator gets blocked in the line before "value=()".
        
        # Next execution cycle c(n+1) resumes upon send(value(n+1)) in the line.
        # "value=()" is executed with the new context from send(value(n)) at t=n+1.
        # Hence the value is from the new cycle c(n+1).
        value = (yield event_expression)
        print("After yield. event_expression is {} value is {}".format(event_expression, value))

        event_expression = value if value else event_expression
        print("After assignment. event_expression is {} value is {}".format(event_expression, value))
        
        # c(n+1) continues to the "yield event_expression" expression and returns the evaluation result. 
        
# Generator is instantiated but none executed in it.
g = f()

# Execution cycle c(t=0). 
# send(None) moves the execution to the "yield event_expression" expression.
# event_expression=99999 is evaluated and returned to the caller.
# c(0) ends here and the generator get blocked in the line without executing "value=()" statement.
# You must separate "yield ..." and "value=()" although they are in the same line.
# "yield" is within c(0) before blocking. "value=()" is in the next c(1) after the resume.
print("1: send(None) is {}\n".format(g.send(None)))

# Execution cycle c(1). 
# send(5) resumes the generator in the same yield statement but at the assignment "value=()".
# A new context from send(5) is framed and "value=(5)" is executed.
# c(1) continues to the "yield ..." expression and event_expression=5 is evaluated and returned to the caller.
# c(1) ends here and the generator get blocked in the line without executing "value=()" statement.
print("2: calling send(5)")
print("2: send(5) returned {}\n".format(g.send(5)))

print("3: calling next(g)")
print("3: next(g) returned {}".format(next(g)))

1: send(None) is 99999

2: calling send(5)
After yield. event_expression is 99999 value is 5
After assignment. event_expression is 5 value is 5
2: send(5) returned 5

3: calling next(g)
After yield. event_expression is 5 value is None
After assignment. event_expression is 5 value is None
3: next(g) returned 5


In [40]:
def g():
    event = 99999
    while True:
        print("g() started and is before yield statement.")
        
        # https://stackoverflow.com/a/36997251/4281353
        # Whenever a generator reaches a yield expression, execution pauses right there, 
        # the expression can't produce anything inside the generator until it is resumed.
        value = (yield event)

        print("Resumed after yield. event is {} value is {}".format(event, value))
        event = value if value else event

        print("Update. event is {} value is {}".format(event))
        
def f():
    exception_value = "hoge"
    while True:
        yield g()
        return exception_value  # Same with raising StopIteration(exception_value)

_g = next(f())

In [41]:
print("1: send(None) is {}\n".format(_g.send(None)))

g() started and is before yield statement.
1: send(None) is 99999

